# Application d'analyse des données de Formule 1 saison 2022
celle que je peux récupérer en tout cas

Cette application est réutilisable pour les prochaines saisons

## Imports

In [2]:
%matplotlib inline
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import json
import xml.etree.cElementTree as et

## Backend
dans cette partie, je fais tous ce qui ne relève pas de l'affichage des graphs.
Cela permet d'avoir directement toutes les données d'un coup pour ensuite juste avoir a
choisir le graph a afficher

In [4]:
# import data
data_result = pd.read_csv('data/race_result.csv', sep=",")
data_points = pd.read_csv('data/race_points.csv', sep=",")
data_times = pd.read_csv('data/race_times.csv', sep=",")
data_best_lap = pd.read_csv('data/race_best_lap.csv', sep=",")

X = [i for i in range(1, data_result.shape[1]+1)]

jsonfile = open("teams.json")
teams_pilot = json.load(jsonfile)

# get each pilot's data
pilots_data = {
    "ham": [[] for i in range(4)],
    "rus": [[] for i in range(4)],
    "ver": [[] for i in range(4)],
    "per": [[] for i in range(4)],
    "lec": [[] for i in range(4)],
    "sai": [[] for i in range(4)],
    "gas": [[] for i in range(4)],
    "tsu": [[] for i in range(4)],
    "ric": [[] for i in range(4)],
    "nor": [[] for i in range(4)],
    "oco": [[] for i in range(4)],
    "alo": [[] for i in range(4)],
    "vet": [[] for i in range(4)],
    "str": [[] for i in range(4)],
    "lat": [[] for i in range(4)],
    "alb": [[] for i in range(4)],
    "bot": [[] for i in range(4)],
    "zho": [[] for i in range(4)],
    "mag": [[] for i in range(4)],
    "sch": [[] for i in range(4)]
}

for pilot in pilots_data.keys():
    pilots_data[pilot][0] = data_result[pilot]
    pilots_data[pilot][1] = data_points[pilot]
    pilots_data[pilot][2] = data_times[pilot]
    pilots_data[pilot][3] = data_best_lap[pilot]
        


## Frontend
Ici les affichages seront séparé pour pouvoir choisir ce qu'on affiche

### Test data import

In [5]:
print(data_result)
print(data_points)
print(data_times)
print(data_best_lap)

print(pilots_data)

print(teams_pilot)

Empty DataFrame
Columns: [location, date, ham, rus, ver, per, lec, sai, gas, tsu, ric, nor, oco, alo, vet, str, lat, alb, bot, zho, mag, sch]
Index: []

[0 rows x 22 columns]
Empty DataFrame
Columns: [location, date, ham, rus, ver, per, lec, sai, gas, tsu, ric, nor, oco, alo, vet, str, lat, alb, bot, zho, mag, sch]
Index: []

[0 rows x 22 columns]
Empty DataFrame
Columns: [location, date, ham, rus, ver, per, lec, sai, gas, tsu, ric, nor, oco, alo, vet, str, lat, alb, bot, zho, mag, sch]
Index: []

[0 rows x 22 columns]
Empty DataFrame
Columns: [location, date, ham, rus, ver, per, lec, sai, gas, tsu, ric, nor, oco, alo, vet, str, lat, alb, bot, zho, mag, sch]
Index: []

[0 rows x 22 columns]
{'ham': [Series([], Name: ham, dtype: object), Series([], Name: ham, dtype: object), Series([], Name: ham, dtype: object), Series([], Name: ham, dtype: object)], 'rus': [Series([], Name: rus, dtype: object), Series([], Name: rus, dtype: object), Series([], Name: rus, dtype: object), Series([], Name: